In [1]:
#NaN補完後、NaNリカバリー
# === 必要なライブラリ ===
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from tqdm import tqdm
from pathlib import Path

# === 2指数関数モデル定義 ===
def double_exp_independent(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === NaN復元ありSG + 褪色補正処理関数 ===
def process_signal_preserve_nan(df_raw, label):
    df_filtered = pd.DataFrame()
    df_corrected = pd.DataFrame()
    t = np.arange(len(df_raw))

    # 初期値: 全cellの1フレーム目の平均
    first_frame_vals = df_raw.iloc[0, 1:].dropna()
    a0 = c0 = first_frame_vals.mean()

    for col in tqdm(df_raw.columns[1:], desc=f"Processing {label} (preserve NaN)"):
        y = df_raw[col].astype(float).values
        nan_mask = np.isnan(y)

        if np.all(nan_mask):
            df_filtered[col] = y
            df_corrected[col] = y
            continue

        # 一時的に補完（線形補間 + 前後補完）
        y_interp = pd.Series(y).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values

        # SGフィルタ
        y_sg = savgol_filter(y_interp, window_length=7, polyorder=1)
        y_sg[nan_mask] = np.nan  # 元のNaNを復元
        df_filtered[col] = y_sg

        try:
            # 褪色補正（NaN除いたデータでフィット）
            popt, _ = curve_fit(
                double_exp_independent,
                t[~nan_mask],
                y_sg[~nan_mask],
                p0=[a0, 0.01, c0, 0.001],
                bounds=([0, 0, 0, 0], [2000, 1, 2000, 1]),
                maxfev=10000
            )
            fitted = double_exp_independent(t, *popt)
            scale = double_exp_independent(0, *popt) / fitted
            corrected = y_sg * scale
            corrected[nan_mask] = np.nan  # 元のNaNを復元
            df_corrected[col] = corrected
        except RuntimeError:
            df_corrected[col] = y_sg  # フィット失敗時はSG済みデータのみ使用

    # timing列を復元
    df_filtered.insert(0, 'timing', df_raw['timing'])
    df_corrected.insert(0, 'timing', df_raw['timing'])
    return df_filtered, df_corrected

# === 対象ファイルとラベルのペア ===
input_files = [
    ("ensemble_tdTomato_10mW-4_v2_cleaned.csv", "tdTomato_10mW-4_v2"),
    ("ensemble_GCaMP_10mW-4_v2_cleaned.csv", "GCaMP_10mW-4_v2"),
    ("ensemble_tdTomato_12mW-3_v2_cleaned.csv", "tdTomato_12mW-3_v2"),
    ("ensemble_GCaMP_12mW-3_v2_cleaned.csv", "GCaMP_12mW-3_v2"),
    ("ensemble_tdTomato_7mW-3_cleaned.csv", "tdTomato_7mW-3"),
    ("ensemble_GCaMP_7mW-3_cleaned.csv", "GCaMP_7mW-3"),
]

# === 出力ディレクトリの作成 ===
output_dir = Path("bleach_corrected_preserve_nan")
output_dir.mkdir(parents=True, exist_ok=True)

# === 一括処理ループ ===
for filename, label in input_files:
    print(f"\n=== Processing: {filename} ===")
    df = pd.read_csv(filename)

    # SG + 褪色補正（NaN補完 → 処理 → NaN復元）
    df_filtered, df_corrected = process_signal_preserve_nan(df, label)

    # 出力ファイル保存
    df_filtered.to_csv(output_dir / f"{label}_sg_filtered_win7.csv", index=False)
    df_corrected.to_csv(output_dir / f"{label}_bleach_corrected_win7.csv", index=False)

print("\n✅ 全データの処理と保存が完了しました。")



=== Processing: ensemble_tdTomato_10mW-4_v2_cleaned.csv ===


Processing tdTomato_10mW-4_v2 (preserve NaN): 100%|█| 52/52 [00:01<00:00, 27.65i



=== Processing: ensemble_GCaMP_10mW-4_v2_cleaned.csv ===


Processing GCaMP_10mW-4_v2 (preserve NaN): 100%|█| 52/52 [00:01<00:00, 37.15it/s



=== Processing: ensemble_tdTomato_12mW-3_v2_cleaned.csv ===


...


=== Processing: ensemble_tdTomato_7mW-3_cleaned.csv ===


Processing tdTomato_7mW-3 (preserve NaN): 100%|█| 72/72 [00:01<00:00, 53.82it/s]



=== Processing: ensemble_GCaMP_7mW-3_cleaned.csv ===


Processing GCaMP_7mW-3 (preserve NaN): 100%|████| 72/72 [00:02<00:00, 29.86it/s]



✅ 全データの処理と保存が完了しました。


In [1]:
# === 必要なライブラリ ===
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from tqdm import tqdm
from pathlib import Path

# === 2指数関数モデル定義 ===
def double_exp_independent(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === SG + 褪色補正処理関数 ===
def process_signal(df_raw, label):
    df_filtered = pd.DataFrame()
    df_corrected = pd.DataFrame()
    t = np.arange(len(df_raw))

    # 初期値: 全cellの1フレーム目の平均
    first_frame_vals = df_raw.iloc[0, 1:].dropna()
    a0 = c0 = first_frame_vals.mean()

    for col in tqdm(df_raw.columns[1:], desc=f"Processing {label}"):
        y = df_raw[col].astype(float).values
        if np.isnan(y).all():
            df_filtered[col] = y
            df_corrected[col] = y
            continue

        # NaN補完（前後補間 + 端補完）
        y_interp = pd.Series(y).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values

        # SGフィルタ適用
        y_sg = savgol_filter(y_interp, window_length=13, polyorder=1)
        df_filtered[col] = y_sg

        try:
            # 褪色補正フィット
            popt, _ = curve_fit(
                double_exp_independent,
                t,
                y_sg,
                p0=[a0, 0.01, c0, 0.001],
                bounds=([0, 0, 0, 0], [2000, 1, 2000, 1]),
                maxfev=10000
            )
            fitted = double_exp_independent(t, *popt)
            scale = double_exp_independent(0, *popt) / fitted
            corrected = y_sg * scale
            df_corrected[col] = corrected
        except RuntimeError:
            df_corrected[col] = y_sg  # フィット失敗時はSG済みデータ

    # timing列を復元
    df_filtered.insert(0, 'timing', df_raw['timing'])
    df_corrected.insert(0, 'timing', df_raw['timing'])
    return df_filtered, df_corrected

# === 対象ファイルとラベルのペア ===
input_files = [
    ("ensemble_tdTomato_7mW-3_cleaned.csv", "tdTomato_7mW-3"),
    ("ensemble_GCaMP_7mW-3_cleaned.csv", "GCaMP_7mW-3"),
    ("ensemble_tdTomato_10mW-4_cleaned.csv", "tdTomato_10mW-4"),
    ("ensemble_GCaMP_10mW-4_cleaned.csv", "GCaMP_10mW-4"),
    ("ensemble_tdTomato_12mW-3_cleaned.csv", "tdTomato_12mW-3"),
    ("ensemble_GCaMP_12mW-3_cleaned.csv", "GCaMP_12mW-3"),
]

# === 出力ディレクトリの作成 ===
output_dir = Path("bleach_corrected_all_v2")
output_dir.mkdir(parents=True, exist_ok=True)

# === 一括処理ループ ===
for filename, label in input_files:
    print(f"\n=== Processing: {filename} ===")
    df = pd.read_csv(filename)

    # SG + 褪色補正
    df_filtered, df_corrected = process_signal(df, label)

    # 出力ファイル保存
    df_filtered.to_csv(output_dir / f"{label}_sg_filtered.csv", index=False)
    df_corrected.to_csv(output_dir / f"{label}_bleach_corrected.csv", index=False)

print("\n✅ 全データの処理と保存が完了しました。")



=== Processing: ensemble_tdTomato_7mW-3_cleaned.csv ===


Processing tdTomato_7mW-3: 100%|████████████████| 72/72 [00:01<00:00, 53.22it/s]



=== Processing: ensemble_GCaMP_7mW-3_cleaned.csv ===


Processing GCaMP_7mW-3: 100%|███████████████████| 72/72 [00:01<00:00, 50.87it/s]



=== Processing: ensemble_tdTomato_10mW-4_cleaned.csv ===


...


=== Processing: ensemble_tdTomato_12mW-3_cleaned.csv ===


Processing tdTomato_12mW-3: 100%|███████████████| 30/30 [00:00<00:00, 60.28it/s]



=== Processing: ensemble_GCaMP_12mW-3_cleaned.csv ===


Processing GCaMP_12mW-3: 100%|██████████████████| 30/30 [00:00<00:00, 71.27it/s]



✅ 全データの処理と保存が完了しました。


In [5]:
# === 必要なライブラリ ===
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from tqdm import tqdm
from pathlib import Path

# === 2指数関数モデル定義 ===
def double_exp_independent(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === SG + 褪色補正処理関数 ===
def process_signal(df_raw, label):
    df_filtered = pd.DataFrame()
    df_corrected = pd.DataFrame()
    t = np.arange(len(df_raw))

    # 初期値: 全cellの1フレーム目の平均
    first_frame_vals = df_raw.iloc[0, 1:].dropna()
    a0 = c0 = first_frame_vals.mean()

    for col in tqdm(df_raw.columns[1:], desc=f"Processing {label}"):
        y = df_raw[col].astype(float).values
        if np.isnan(y).all():
            df_filtered[col] = y
            df_corrected[col] = y
            continue

        # NaN補完（前後補間 + 端補完）
        y_interp = pd.Series(y).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values

        # SGフィルタ適用
        y_sg = savgol_filter(y_interp, window_length=13, polyorder=1)
        df_filtered[col] = y_sg

        try:
            # 褪色補正フィット
            popt, _ = curve_fit(
                double_exp_independent,
                t,
                y_sg,
                p0=[a0, 0.01, c0, 0.001],
                bounds=([0, 0, 0, 0], [2000, 1, 2000, 1]),
                maxfev=10000
            )
            fitted = double_exp_independent(t, *popt)
            scale = double_exp_independent(0, *popt) / fitted
            corrected = y_sg * scale
            df_corrected[col] = corrected
        except RuntimeError:
            df_corrected[col] = y_sg  # フィット失敗時はSG済みデータ

    # timing列を復元
    df_filtered.insert(0, 'timing', df_raw['timing'])
    df_corrected.insert(0, 'timing', df_raw['timing'])
    return df_filtered, df_corrected

# === 対象ファイルとラベルのペア ===
input_files = [
    ("ensemble_tdTomato_cleaned_10mW-1_good.csv", "tdTomato_cleaned_10mW-1_good"),
    ("ensemble_GCaMP_cleaned_10mW-1_good.csv", "GCaMP_cleaned_10mW-1_good"),

]

# === 出力ディレクトリの作成 ===
output_dir = Path("bleach_corrected_all_v2")
output_dir.mkdir(parents=True, exist_ok=True)

# === 一括処理ループ ===
for filename, label in input_files:
    print(f"\n=== Processing: {filename} ===")
    df = pd.read_csv(filename)

    # SG + 褪色補正
    df_filtered, df_corrected = process_signal(df, label)

    # 出力ファイル保存
    df_filtered.to_csv(output_dir / f"{label}_sg_filtered.csv", index=False)
    df_corrected.to_csv(output_dir / f"{label}_bleach_corrected.csv", index=False)

print("\n✅ 全データの処理と保存が完了しました。")



=== Processing: ensemble_tdTomato_cleaned_10mW-1_good.csv ===


Processing tdTomato_cleaned_10mW-1_good: 100%|██| 93/93 [00:02<00:00, 43.53it/s]



=== Processing: ensemble_GCaMP_cleaned_10mW-1_good.csv ===


Processing GCaMP_cleaned_10mW-1_good: 100%|█████| 93/93 [00:02<00:00, 44.94it/s]



✅ 全データの処理と保存が完了しました。


In [ ]:
#NaN補完後、NaNリカバリー
# === 必要なライブラリ ===
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
from tqdm import tqdm
from pathlib import Path

# === 2指数関数モデル定義 ===
def double_exp_independent(t, a, b, c, d):
    return a * np.exp(-b * t) + c * np.exp(-d * t)

# === NaN復元ありSG + 褪色補正処理関数 ===
def process_signal_preserve_nan(df_raw, label):
    df_filtered = pd.DataFrame()
    df_corrected = pd.DataFrame()
    t = np.arange(len(df_raw))

    # 初期値: 全cellの1フレーム目の平均
    first_frame_vals = df_raw.iloc[0, 1:].dropna()
    a0 = c0 = first_frame_vals.mean()

    for col in tqdm(df_raw.columns[1:], desc=f"Processing {label} (preserve NaN)"):
        y = df_raw[col].astype(float).values
        nan_mask = np.isnan(y)

        if np.all(nan_mask):
            df_filtered[col] = y
            df_corrected[col] = y
            continue

        # 一時的に補完（線形補間 + 前後補完）
        y_interp = pd.Series(y).interpolate(limit_direction='both').fillna(method='bfill').fillna(method='ffill').values

        # SGフィルタ
        y_sg = savgol_filter(y_interp, window_length=13, polyorder=1)
        y_sg[nan_mask] = np.nan  # 元のNaNを復元
        df_filtered[col] = y_sg

        try:
            # 褪色補正（NaN除いたデータでフィット）
            popt, _ = curve_fit(
                double_exp_independent,
                t[~nan_mask],
                y_sg[~nan_mask],
                p0=[a0, 0.01, c0, 0.001],
                bounds=([0, 0, 0, 0], [2000, 1, 2000, 1]),
                maxfev=10000
            )
            fitted = double_exp_independent(t, *popt)
            scale = double_exp_independent(0, *popt) / fitted
            corrected = y_sg * scale
            corrected[nan_mask] = np.nan  # 元のNaNを復元
            df_corrected[col] = corrected
        except RuntimeError:
            df_corrected[col] = y_sg  # フィット失敗時はSG済みデータのみ使用

    # timing列を復元
    df_filtered.insert(0, 'timing', df_raw['timing'])
    df_corrected.insert(0, 'timing', df_raw['timing'])
    return df_filtered, df_corrected

# === 対象ファイルとラベルのペア ===
input_files = [
    ("ensemble_tdTomato_7mW-3_cleaned.csv", "tdTomato_7mW-3"),
    ("ensemble_GCaMP_7mW-3_cleaned.csv", "GCaMP_7mW-3"),
    ("ensemble_tdTomato_10mW-4_cleaned.csv", "tdTomato_10mW-4"),
    ("ensemble_GCaMP_10mW-4_cleaned.csv", "GCaMP_10mW-4"),
    ("ensemble_tdTomato_12mW-3_cleaned.csv", "tdTomato_12mW-3"),
    ("ensemble_GCaMP_12mW-3_cleaned.csv", "GCaMP_12mW-3"),
]

# === 出力ディレクトリの作成 ===
output_dir = Path("bleach_corrected_preserve_nan")
output_dir.mkdir(parents=True, exist_ok=True)

# === 一括処理ループ ===
for filename, label in input_files:
    print(f"\n=== Processing: {filename} ===")
    df = pd.read_csv(filename)

    # SG + 褪色補正（NaN補完 → 処理 → NaN復元）
    df_filtered, df_corrected = process_signal_preserve_nan(df, label)

    # 出力ファイル保存
    df_filtered.to_csv(output_dir / f"{label}_sg_filtered.csv", index=False)
    df_corrected.to_csv(output_dir / f"{label}_bleach_corrected.csv", index=False)

print("\n✅ 全データの処理と保存が完了しました。")
